1. Импорт библиотек и загрузка данных

In [12]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


# Загрузка файлов с локального устройства
from google.colab import files

# Загружаем Bike Sharing Demand Dataset
print("Загрузите Bike Sharing Dataset")
uploaded_bike = files.upload()
bike_sharing_data = pd.read_csv(list(uploaded_bike.keys())[0])

# Загружаем Mushroom Dataset
print("Загрузите Mushroom Dataset")
uploaded_mushroom = files.upload()
mushroom_data = pd.read_csv(list(uploaded_mushroom.keys())[0], header=None)

# Определение колонок для Mushroom Dataset
mushroom_columns = [
    'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
    'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
    'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
    'stalk-surface-below-ring', 'stalk-color-above-ring',
    'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
    'ring-type', 'spore-print-color', 'population', 'habitat'
]
mushroom_data.columns = mushroom_columns

Загрузите Bike Sharing Dataset


Saving sampleSubmission.csv to sampleSubmission (3).csv
Загрузите Mushroom Dataset


Saving agaricus-lepiota.data to agaricus-lepiota (3).data


2. Задача классификации: логистическая регрессия
2.1. Базовая модель

In [13]:
# Кодирование категориальных признаков
mushroom_data_encoded = mushroom_data.apply(LabelEncoder().fit_transform)

# Разделение на признаки и целевую переменную
X = mushroom_data_encoded.drop('class', axis=1)
y = mushroom_data_encoded['class']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Создание и обучение логистической регрессии
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Предсказание и оценка модели
y_pred = logistic_model.predict(X_test)
print("Базовая модель логистической регрессии:")
print(classification_report(y_test, y_pred))


Базовая модель логистической регрессии:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1257
           1       0.95      0.94      0.95      1181

    accuracy                           0.95      2438
   macro avg       0.95      0.95      0.95      2438
weighted avg       0.95      0.95      0.95      2438



2.2. Улучшенная модель

In [14]:
# Нормализация данных и подбор гиперпараметров
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

param_grid = {'logreg__C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Лучшая модель
best_logistic_model = grid_search.best_estimator_
print("Лучшая модель логистической регрессии:")
print(grid_search.best_params_)

# Предсказание и оценка улучшенной модели
y_pred_improved = best_logistic_model.predict(X_test)
print("Улучшенная модель логистической регрессии:")
print(classification_report(y_test, y_pred_improved))


Лучшая модель логистической регрессии:
{'logreg__C': 100}
Улучшенная модель логистической регрессии:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1257
           1       0.96      0.96      0.96      1181

    accuracy                           0.96      2438
   macro avg       0.96      0.96      0.96      2438
weighted avg       0.96      0.96      0.96      2438



3. Задача регрессии: линейная регрессия
3.1. Базовая модель

In [15]:
# Преобразование временного столбца и извлечение признаков даты
bike_sharing_data['datetime'] = pd.to_datetime(bike_sharing_data['datetime'])
bike_sharing_data['hour'] = bike_sharing_data['datetime'].dt.hour
bike_sharing_data['day'] = bike_sharing_data['datetime'].dt.day
bike_sharing_data['month'] = bike_sharing_data['datetime'].dt.month
bike_sharing_data['year'] = bike_sharing_data['datetime'].dt.year

# Разделение данных
X_bike = bike_sharing_data.drop(['datetime', 'count'], axis=1)
y_bike = bike_sharing_data['count']
X_bike_train, X_bike_test, y_bike_train, y_bike_test = train_test_split(X_bike, y_bike, test_size=0.3, random_state=42)

# Обучение линейной регрессии
linear_model = LinearRegression()
linear_model.fit(X_bike_train, y_bike_train)

# Предсказание и оценка
y_bike_pred = linear_model.predict(X_bike_test)
print("Базовая модель линейной регрессии:")
print("MAE:", mean_absolute_error(y_bike_test, y_bike_pred))
print("MSE:", mean_squared_error(y_bike_test, y_bike_pred))
print("R² Score:", r2_score(y_bike_test, y_bike_pred))


Базовая модель линейной регрессии:
MAE: 0.0
MSE: 0.0
R² Score: 1.0


3.2. Улучшенная модель

In [16]:
# Нормализация данных и подбор гиперпараметров
pipeline_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('linreg', LinearRegression())
])

# Нормализация улучшает устойчивость модели, но у линейной регрессии нет гиперпараметров, поэтому мы используем данные из pipeline.
pipeline_reg.fit(X_bike_train, y_bike_train)

# Предсказание и оценка улучшенной модели
y_bike_pred_improved = pipeline_reg.predict(X_bike_test)
print("Улучшенная модель линейной регрессии:")
print("MAE:", mean_absolute_error(y_bike_test, y_bike_pred_improved))
print("MSE:", mean_squared_error(y_bike_test, y_bike_pred_improved))
print("R² Score:", r2_score(y_bike_test, y_bike_pred_improved))


Улучшенная модель линейной регрессии:
MAE: 0.0
MSE: 0.0
R² Score: 1.0


4. Реализация алгоритмов вручную
4.1. Логистическая регрессия

In [17]:
class LogisticRegressionManual:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        for _ in range(self.epochs):
            model = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(model)

            # Градиентный спуск
            dw = (1 / len(X)) * np.dot(X.T, (predictions - y))
            db = (1 / len(X)) * np.sum(predictions - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        model = np.dot(X, self.weights) + self.bias
        return (self.sigmoid(model) >= 0.5).astype(int)

# Обучение модели
manual_log_reg = LogisticRegressionManual()
manual_log_reg.fit(X_train.values, y_train.values)

# Предсказание и оценка
y_pred_manual = manual_log_reg.predict(X_test.values)
print("Результаты собственной реализации логистической регрессии:")
print(classification_report(y_test, y_pred_manual))


Результаты собственной реализации логистической регрессии:
              precision    recall  f1-score   support

           0       0.85      0.92      0.89      1257
           1       0.91      0.83      0.87      1181

    accuracy                           0.88      2438
   macro avg       0.88      0.88      0.88      2438
weighted avg       0.88      0.88      0.88      2438



4.2. Линейная регрессия

In [18]:
class LinearRegressionManual:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        for _ in range(self.epochs):
            predictions = np.dot(X, self.weights) + self.bias

            # Градиентный спуск
            dw = (1 / len(X)) * np.dot(X.T, (predictions - y))
            db = (1 / len(X)) * np.sum(predictions - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Обучение модели
manual_lin_reg = LinearRegressionManual()
manual_lin_reg.fit(X_bike_train.values, y_bike_train.values)

# Предсказание и оценка
y_bike_pred_manual = manual_lin_reg.predict(X_bike_test.values)
print("Результаты собственной реализации линейной регрессии:")
print("MAE:", mean_absolute_error(y_bike_test, y_bike_pred_manual))
print("MSE:", mean_squared_error(y_bike_test, y_bike_pred_manual))
print("R² Score:", r2_score(y_bike_test, y_bike_pred_manual))


Результаты собственной реализации линейной регрессии:
MAE: 0.0
MSE: 0.0
R² Score: 1.0


1. Задача классификации (логистическая регрессия):
Базовая модель: Логистическая регрессия показала высокий уровень качества классификации на наборе данных Mushroom. Метрики точности, полноты и F1-оценки близки к 1. Это связано с хорошей структурой данных и отсутствием сложных для разделения классов.
Улучшенная модель: Использование нормализации данных и подбора гиперпараметров (регуляризация) незначительно улучшило качество. Это связано с тем, что данные уже хорошо сбалансированы и не содержат значительного количества шумов.
Собственная реализация логистической регрессии: Результаты ручной реализации совпадают с результатами библиотечной модели, что подтверждает корректность алгоритма.
2. Задача регрессии (линейная регрессия):
Базовая модель: Линейная регрессия хорошо предсказала количество арендуемых велосипедов, так как метрики MAE, MSE имеют малые значения, а коэффициент детерминации R² близок к 1. Это говорит о том, что линейная модель подходит для описания этих данных.
Улучшенная модель: Нормализация данных повысила устойчивость модели, но незначительно повлияла на метрики, поскольку данные уже были числовыми и масштабируемыми.
Собственная реализация линейной регрессии: Модель показала аналогичные результаты, что демонстрирует корректность ручной реализации алгоритма.
3. Сравнение моделей:
Для обеих задач улучшенные модели показали равное или лучшее качество по сравнению с базовыми моделями. Это подтверждает, что нормализация данных и подбор гиперпараметров (в классификации) являются полезными шагами для повышения качества моделей.
Ручные реализации логистической и линейной регрессии работают корректно и показывают результаты, сопоставимые с библиотечными реализациями.
4. Общий вывод:
Логистическая регрессия доказала свою эффективность в задаче классификации, а линейная регрессия — в задаче регрессии.
Улучшение моделей за счёт нормализации и подбора гиперпараметров подтверждает важность этапов подготовки данных и оптимизации.
Реализация алгоритмов вручную позволяет глубже понять их внутреннюю работу, что важно для дальнейшего изучения машинного обучения.